# Example Notebook

In [ ]:
import json 
from pathlib import Path
import shutil

from wattile.entry_point import init_logging
from wattile.data_reading import read_dataset_from_file
from wattile.buildings_processing import prep_for_rnn
from wattile.models import AlfaModel


PROJECT_DIRECTORY = Path().resolve().parents[1]


In [ ]:
"""
For this example, we will be using the default configs.
Check out the docs for an explaination of each config.
"""
with open(PROJECT_DIRECTORY / "wattile" / "configs" / "configs.json", "r") as f:
    configs = json.load(f)

exp_dir = PROJECT_DIRECTORY / "notebooks" / "exp_dir"
if exp_dir.exists():
    shutil.rmtree(exp_dir)
exp_dir.mkdir()

configs["data_output"]["exp_dir"] = str(exp_dir)
configs["data_input"]["data_dir"] = str(PROJECT_DIRECTORY / "tests" / "data" / "Synthetic Site")

configs

In [ ]:
"""
Firstly, we will read the raw data from the dataset. 
Checkout the docs for an indepth explaination of necessary dataset structure.
"""
data = read_dataset_from_file(configs)
data


In [ ]:
"""
Next, we call `prep_for_rnn` do preform some data preprocessing.
"""
train_df, val_df = prep_for_rnn(configs, data)
train_df

In [ ]:
"""
Finally, we are ready to train our model!
"""
init_logging(local_results_dir=configs["data_output"]["exp_dir"])
model = AlfaModel(configs)
model.train(train_df, val_df)

In [ ]:
f"""
See {exp_dir} for the results.
"""